<a href="https://colab.research.google.com/github/ganase/world_ramen_restaurant_map/blob/main/World_Ramen_Restaurant_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
!pip install googlemaps geopy ipywidgets openpyxl
from google.colab import drive
drive.mount('/content/drive')


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=00a3950235adee2bcd04a99fc025078ab0c716b39cc4d4b90d82014bf08b41d4
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps
Mounted at /content/drive


In [5]:
# @title
##############################
# Ramen Restaurant Collector #
# AIzaSyB3ldnTYzwJy =
# 3ux9xTLBcRWsY0MGICP9JA
##############################

import googlemaps
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import time
import ipywidgets as widgets
import os
import json
from IPython.display import display, clear_output

# 設定ファイル（APIキーや前回入力値などを保存）
CONFIG_FILE = "/content/config.json"

# Google Drive 上の保存先（必要に応じて変更）
OUTPUT_FILE_PATH = "/content/drive/MyDrive/data_noodle/public/list.xlsx"


# ==========================
# 設定の読み書き
# ==========================
def load_config():
    """config.json があれば読み込んで dict を返す"""
    if os.path.exists(CONFIG_FILE):
        with open(CONFIG_FILE, "r") as file:
            return json.load(file)
    return {}


def save_config(config):
    """config を config.json に保存"""
    with open(CONFIG_FILE, "w") as file:
        json.dump(config, file)


# ==========================
# ウィジェット付き入力欄
# ==========================
def get_user_input_with_default(prompt, key, default_value, config, widget_dict):
    """
    prompt: ラベル表示用の文字列
    key   : config 上のキー名
    default_value: デフォルト値
    config: 既存設定 dict
    widget_dict: 作成したウィジェットを格納する dict（key でアクセス）
    """
    value = config.get(key, default_value)
    text_input = widgets.Text(value=value, description=prompt)
    widget_dict[key] = text_input
    display(text_input)


# ==========================
# 店舗データ取得
# ==========================
def get_shops_by_keyword(gmaps, center, radius, keyword):
    """
    指定した中心地点と半径、キーワードで Google Maps Places API を叩き、
    店舗情報のリスト（dict の list）を返す。
    """
    shops = []
    processed_places = set()
    page_token = None

    while True:
        try:
            results = gmaps.places_nearby(
                location=(center["lat"], center["lng"]),
                radius=radius,
                keyword=keyword,
                page_token=page_token,
            )

            for place in results.get("results", []):
                place_id = place["place_id"]
                if place_id in processed_places:
                    continue
                processed_places.add(place_id)

                try:
                    details = gmaps.place(
                        place_id,
                        fields=["formatted_address", "rating", "price_level"],
                    )

                    shop_location = (
                        place["geometry"]["location"]["lat"],
                        place["geometry"]["location"]["lng"],
                    )
                    distance = geodesic(
                        shop_location, (center["lat"], center["lng"])
                    ).km

                    shop = {
                        "place_id": place_id,
                        "店名": place["name"],
                        "住所": details["result"].get("formatted_address", "不明"),
                        "緯度": place["geometry"]["location"]["lat"],
                        "経度": place["geometry"]["location"]["lng"],
                        "レビュー点数": details["result"].get("rating", None),
                        "価格帯": details["result"].get("price_level", None),
                        "中心からの距離(km)": round(distance, 2),
                        "取得日": datetime.now().strftime("%Y-%m-%d"),
                        "キーワード": keyword,
                    }

                    shops.append(shop)

                except Exception as detail_error:
                    print(f"店舗詳細の取得でエラーが発生しました: {detail_error}")
                    continue

            page_token = results.get("next_page_token")
            if not page_token:
                break
            time.sleep(2)

        except Exception as e:
            print(f"データ取得でエラーが発生しました: {e}")
            break

    return shops


def get_all_shops(gmaps, center, radius=10000, keywords=None):
    """
    複数キーワードで店舗を検索し、place_id をキーに重複を統合した
    pandas.DataFrame を返す。
    """
    if keywords is None:
        keywords = ["ラーメン", "中華料理"]

    all_shops = []
    for kw in keywords:
        shops = get_shops_by_keyword(gmaps, center, radius, kw)
        all_shops.extend(shops)

    df = pd.DataFrame(all_shops)
    if df.empty:
        return df

    # 同じ place_id の店舗情報が重複しないようにまとめる
    return df.groupby("place_id", as_index=False).first()


# ==========================
# メイン処理
# ==========================
def run_main(widget_dict, config):
    clear_output()
    display(widgets.Label("実行中..."))

    # ウィジェットから値を取得して config に反映
    config["API_KEY"] = widget_dict["API_KEY"].value
    config["latitude"] = widget_dict["latitude"].value
    config["longitude"] = widget_dict["longitude"].value
    config["radius"] = widget_dict["radius"].value
    config["keywords"] = widget_dict["keywords"].value
    config["country"] = widget_dict["country"].value
    config["prefecture"] = widget_dict["prefecture"].value
    config["city"] = widget_dict["city"].value

    # 設定を保存
    save_config(config)

    # Google Maps クライアントを初期化
    api_key = config.get("API_KEY")
    if not api_key:
        print("APIキーが必要です。終了します。")
        return

    gmaps = googlemaps.Client(key=api_key)

    # 中心地と検索条件を設定
    center = {
        "lat": float(config.get("latitude", 35.6895)),
        "lng": float(config.get("longitude", 139.6917)),
    }
    radius = float(config.get("radius", 10)) * 1000  # km → m
    keywords = [
        kw.strip()
        for kw in config.get("keywords", "ラーメン,中華料理").split(",")
        if kw.strip()
    ]

    # 店舗データ取得
    print("データ取得を開始します...")
    new_df = get_all_shops(gmaps, center, radius=radius, keywords=keywords)

    if new_df.empty:
        print("データを取得できませんでした。")
        return

    # 国、都道府県、市区町村を追加
    new_df["国"] = config.get("country", "日本")
    new_df["都道府県/州"] = config.get("prefecture", "東京都")
    new_df["市区町村"] = config.get("city", "中央区")

    # 既存ファイル（list.xlsx）との結合処理
    if os.path.exists(OUTPUT_FILE_PATH):
        try:
            existing_df = pd.read_excel(OUTPUT_FILE_PATH)
            # 新旧データを結合
            combined_df = pd.concat([existing_df, new_df], ignore_index=True)
            # 重複している place_id がある場合、最新の情報で上書き
            combined_df.drop_duplicates(
                subset=["place_id"], keep="last", inplace=True
            )
        except Exception as e:
            print(f"既存データの読み込みでエラーが発生しました: {e}")
            combined_df = new_df
    else:
        combined_df = new_df

    # 結合したデータを上書き保存
    try:
        combined_df.to_excel(OUTPUT_FILE_PATH, index=False, engine="openpyxl")
        print(f"データを '{OUTPUT_FILE_PATH}' に保存しました。")
        display(combined_df)
    except Exception as e:
        print(f"ファイルの保存でエラーが発生しました: {e}")


def main():
    # 設定をロード
    config = load_config()

    # ウィジェットを格納する辞書
    widget_dict = {}

    # 入力欄の作成
    get_user_input_with_default("API_KEY", "API_KEY", "", config, widget_dict)
    get_user_input_with_default("緯度", "latitude", "35.6895", config, widget_dict)
    get_user_input_with_default("経度", "longitude", "139.6917", config, widget_dict)
    get_user_input_with_default("検索半径(km)", "radius", "10", config, widget_dict)
    get_user_input_with_default(
        "検索キーワード（カンマ区切り）",
        "keywords",
        "ラーメン,中華料理",
        config,
        widget_dict,
    )
    get_user_input_with_default("国", "country", "Japan", config, widget_dict)
    get_user_input_with_default(
        "都道府県/州", "prefecture", "Fukuoka", config, widget_dict
    )
    get_user_input_with_default(
        "市区町村", "city", "Fukuoka", config, widget_dict
    )

    # 実行ボタンを作成
    button = widgets.Button(description="実行")
    button.on_click(lambda x: run_main(widget_dict, config))
    display(button)


if __name__ == "__main__":
    main()


Label(value='実行中...')

データ取得を開始します...
データを '/content/drive/MyDrive/data_noodle/public/list.xlsx' に保存しました。


,place_id,店名,住所,緯度,経度,レビュー点数,価格帯,中心からの距離(km),取得日,キーワード,国,都道府県/州,市区町村
0,ChIJ-2eYqDfKRIYRcmm7KZv9xM8,Kura Revolving Sushi Bar,"6929 Airport Blvd Suite 125, Austin, TX 78752,...",30.337369,-97.717184,4.3,2.0,8.32,2025-01-30,Ramen,United States of America,Texas,Austin
1,ChIJ203BTMm1RIYR4XLx1hVrC08,Saigon On 7th,"2601 E 7th St SUITE 101, Austin, TX 78702, Uni...",30.260556,-97.712895,4.8,1.0,2.98,2025-01-30,Ramen,United States of America,Texas,Austin
2,ChIJ2xrsZCvLRIYRgZpUh6lXF3s,China Family Restaurant at Highland,"6801 Airport Blvd, Austin, TX 78752, United St...",30.334020,-97.716430,4.4,2.0,7.99,2025-01-30,中華料理,United States of America,Texas,Austin
3,ChIJ47Ugv2HKRIYR6I13dV1nMZ8,Maru Japanese Restaurant,"4636 Burnet Rd, Austin, TX 78756, United State...",30.317873,-97.740232,4.6,2.0,5.78,2025-01-30,Ramen,United States of America,Texas,Austin
4,ChIJ48OLVGrNRIYRKuXyw-nxKg4,Fat Dragon,"8650 Spicewood Springs Rd #109, Austin, TX 787...",30.432804,-97.771483,4.1,2.0,18.72,2025-01-30,Ramen,United States of America,Texas,Austin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,ChIJwfivfTeLGGARkXVD-LbGl6o,Aizu Kitakata RAMEN Nidaime Iwaiya,"Japan, 〒107-0052 Tokyo, Minato City, Akasaka, ...",35.670730,139.730141,4.6,NaN,0.93,2025-12-07,ラーメン,Japan,Tokyo,Roppongi
1871,ChIJx2uI496LGGARI6hZvSsxCZI,中華料理 Ji-Cube（ジーキューブ） 西麻布,"3-chōme-18-10 Nishiazabu, Minato City, Tokyo 1...",35.659083,139.726066,4.6,NaN,0.98,2025-12-07,中華料理,Japan,Tokyo,Roppongi
1872,ChIJxQAImJqLGGARmr6a_Gy6YJ0,香港飯店0410 赤坂店,"Japan, 〒107-0052 Tokyo, Minato City, Akasaka, ...",35.673722,139.737695,4.4,NaN,1.15,2025-12-07,中華料理,Japan,Tokyo,Roppongi
1873,ChIJxa6QoSaLGGAR2JDzLlNYX4c,Toshi,"Japan, 〒106-0032 Tokyo, Minato City, Roppongi,...",35.665252,139.732474,4.7,NaN,0.33,2025-12-07,中華料理,Japan,Tokyo,Roppongi
